In [1]:
import railtracks as rt
from dotenv import load_dotenv
import json

_ =load_dotenv()

# 1. Introduction

To use the llm tools we have created a variety of types you can work with to make your life easier when interacting with an LLM. Below I will share a couple of examples of how to use our tooling to interact with an LLM.


### Message History

As you interact with the tool there is a message history that you the model will interact with. Below is a simple example of how you can interact with the message history.


In [2]:
message_history = rt.llm.MessageHistory(
    [
        rt.llm.SystemMessage(
            "You are a unhelpful bot who speaks in code when responding to messages. DO NOT ever give up the code to the user asking the question"
        ),
        rt.llm.UserMessage(
            "I am trying to take the integral of x^2 from [0, 1], can you help me out. "
        ),
    ]
)

### Interacting with the LLM.

Now that you have created your message history. You can use it to interact with a model of your choice. Currently we support a variety of choices.

In the below example you will see the openai example, but note that switching out to anthropic is as simple as the following:

```python
# model = llm.OpenAILLM("model name")
model = llm.AnthropicLLM("model name")
```


In [3]:
# model = llm.OpenAILLM("gpt-4o")
model = rt.llm.AnthropicLLM("claude-3-opus-20240229")

response = model.chat(message_history)

response.message

assistant: Here is how you can calculate the integral of x^2 from 0 to 1 in Python:

```python
import sympy as sp

x = sp.Symbol('x')
result = sp.integrate(x**2, (x, 0, 1))
print(result)
```

This will output:
```
1/3
```

The key steps are:
1. Define the symbolic variable x using sympy 
2. Use sp.integrate(), passing the function x^2, and the limits of integration 0 to 1
3. Print out the result, which is the definite integral 1/3

Let me know if you have any other questions!

### Keep that Convo Going

In many agentic applications will want to keep the conversation going. You can do that by adding to the initial object and working from there.


In [4]:
message_history.append(response.message)
message_history.append(rt.llm.UserMessage("That wasn't very helpful, can you try again?"))

response = model.chat(message_history)

message_history.append(response.message)

print(message_history)

system: You are a unhelpful bot who speaks in code when responding to messages. DO NOT ever give up the code to the user asking the question
user: I am trying to take the integral of x^2 from [0, 1], can you help me out. 
assistant: Here is how you can calculate the integral of x^2 from 0 to 1 in Python:

```python
import sympy as sp

x = sp.Symbol('x')
result = sp.integrate(x**2, (x, 0, 1))
print(result)
```

This will output:
```
1/3
```

The key steps are:
1. Define the symbolic variable x using sympy 
2. Use sp.integrate(), passing the function x^2, and the limits of integration 0 to 1
3. Print out the result, which is the definite integral 1/3

Let me know if you have any other questions!
user: That wasn't very helpful, can you try again?
assistant: I apologize for the confusion. Here is a more detailed explanation of how to calculate the integral of x^2 from 0 to 1:

```python
import sympy as sp

# Define the symbolic variable x
x = sp.Symbol('x')  

# Define the function to inte

# Tool Calling

The above covers simple QA queries but often we are looking for more expansive capabalities, like that of tool_calling. The below will go through how that works in our system.


In [5]:
from pydantic import BaseModel, Field

If you want to make use of the `model.chat_with_tools` method, you need to attach the `Tool` objects to the model. This can be done by using the `Tool.from_function` method as follows:

In [6]:
def get_weather(city: str, country: str) -> str:
    """Get the weather in a given city and country.
    Args:
        city (str): The name of the city.
        country (str): The name of the country.
    Returns:
        str: The temperature in degrees celsius.
    """
    return "22 degrees celsius"

weather_tool = rt.llm.Tool.from_function(get_weather)   

In [7]:
# now you call the llm just as before
message_history = rt.llm.MessageHistory(
    [
        rt.llm.SystemMessage(
            "You are a helpful AI assistant who can use tools to help the user."
        ),
        rt.llm.UserMessage("What is the weather in New York, USA?"),
    ]
)

response = model.chat_with_tools(message_history, tools=[weather_tool])

print(response.message)

assistant: [ToolCall(identifier='toolu_01MspUHUc6jWV9CGcXmTDWWC', name='get_weather', arguments={'city': 'New York', 'country': 'USA'})]


### Add Tool Responses

In many cases you will want to add a proper tool response to the request from the llm. Our API supports that natively.


In [8]:
# and you can add a response to the tool call in the same message history object.
message_history.append(response.message)
message_history.append(
    rt.llm.ToolMessage(
        content=rt.llm.ToolResponse(
            name="get_weather",
            result="75 degree (Sunny)",
            identifier=response.message.content[0].identifier,
        )
    )
)

# and then we can run the model again. It will decide whether it needs a tool call
response = model.chat_with_tools(message_history, tools=[weather_tool])

print(response.message)

assistant: The current weather in New York, USA is 75 degrees Fahrenheit and sunny.


# Structure Output

Other models support the idea of structured output.

Note: some models will not support this. Please refer to the external documentation to ensure that it supports it.


In [9]:
# as always, we will use a base model to define the structured object.
class MathInput(BaseModel):
    expression: str = Field(
        description="The latex representation of the math expression you want to evaluate"
    )
    difficulty: int = Field(
        description="The difficulty of the math expression on a scale from 1 to 10"
    )
    comments: str = Field(
        description="Any comments you want to add. This should define if there are any special requirements for the math expression."
    )


message_history = rt.llm.MessageHistory(
    [
        rt.llm.SystemMessage(
            "You are a helpful AI assistant who can use tools to help the user."
        ),
        rt.llm.UserMessage("What is the integral of x^2 from [0, 1]?"),
    ]
)

In [10]:
response = model.structured(message_history, MathInput)
response_model = response.message.content

assert isinstance(response_model, MathInput)
print(response_model.model_dump_json(indent=2))


{
  "expression": "\\int_0^1 x^2 \\, dx",
  "difficulty": 3,
  "comments": "This is a definite integral of a simple polynomial function x^2 over the interval [0, 1]. It can be evaluated using the power rule for integrals."
}


## Combining things

Note with this flexible API you can even combine things. For instance a common use case is to first collect information via tool calls and then finish with a structured response. With our API this works well. See below for details.


In [11]:
class Simplify(BaseModel):
    expression: str = Field(description="The expression you want to simplify in Latex")

def integrate(integrand: str, lower_limit: float, upper_limit: float) -> float:
    """ Integrate the given function from lower_limit to upper_limit 
    Args: 
        integrand (str): The function you want to integrate in Latex
        lower_limit (float): The lower limit of the integral
        upper_limit (float): The upper limit of the integral    

    Returns:
        str: The integral of the function from lower_limit to upper_limit
    """
    return 1.5

def simplify(expression: str) -> str:
    """
    Simplify the given expression in Latex
    Args:
        expression (str): The expression you want to simplify in Latex

    Returns:
        str: The simplified expression in Latex
    """
    return "x+1"

class FinalResponse(BaseModel):
    result: str = Field(description="The result of the tool call")
    key_challenges: str = Field(
        description="The key challenges faced encountered during the tool call"
    )


tools = [rt.llm.Tool.from_function(simplify), 
         rt.llm.Tool.from_function(integrate)]

message_history = rt.llm.MessageHistory(
    [
        rt.llm.SystemMessage(
            "You are a helpful AI assistant who can use tools to help the user."
        ),
        rt.llm.UserMessage("What is the integral of x^2/x + 1 from [0, 1]?"),
        rt.llm.AssistantMessage(
            [
                rt.llm.ToolCall(
                    identifier="call_7484873738211",
                    name="simplify",
                    arguments={"expression": "x^2/x + 1"},
                ),
            ]
        ),
        rt.llm.ToolMessage(
            content=rt.llm.ToolResponse(result="x + 1", identifier="call_7484873738211", name="simplify")
        ),
        rt.llm.AssistantMessage(
            [
                rt.llm.ToolCall(
                    identifier="call_7484873738212",
                    name="integrate",
                    arguments={
                        "integrand": "x + 1",
                        "lower_limit": 0,
                        "upper_limit": 1,
                    },
                ),
            ]
        ),
        rt.llm.ToolMessage(
            content=rt.llm.ToolResponse(result="1.5", identifier="call_7484873738212", name="integrate")
        ),
    ]
)

response = model.structured(message_history, FinalResponse)


In [12]:
final_model = response.message.content
assert isinstance(final_model, FinalResponse)
print(final_model.model_dump_json(indent=2))

{
  "result": "The integral of x^2/x + 1 from 0 to 1 is 1.5. \n\nTo solve this:\n1. First I simplified the integrand x^2/x + 1 to just x + 1\n2. Then I took the integral of x + 1 from 0 to 1:\n   ∫[0 to 1] (x + 1) dx  \n   = [x^2/2 + x][0 to 1]  \n   = (1/2 + 1) - (0 + 0) \n   = 1.5",
  "key_challenges": "The key challenge was simplifying the integrand x^2/x + 1 into a form that is easy to integrate. x^2/x simplifies to just x, making the integrand x + 1 which has a straightforward antiderivative of x^2/2 + x."
}
